In [ ]:
print('hello world')

hello world


Hyperparamter => A hyperparameter is a value that you set BEFORE training starts and that controls how the model learns.
  types of hyperparameter .
   1) Gridsearch cv
   2) random cv
   
   3) Bayesian search(important and used)(optuna)

  Optuna code implementation 
   1) objective function
     -> search space
     -> model init
     -> parameter init
     -> training loop
     -> evaluation loop

     return accuracy

    study= trail=10.

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim


In [40]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [41]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [42]:
df = pd.read_csv('Dataset/fmnist_small.csv')
df.head()


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [43]:
# train test split

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    # Convert to PyTorch tensors
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [46]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)


In [47]:
class mynn(nn.Module):
    def __init__(self, input_dim , output_dim ,num_hidden_layers , neureaon_layers , dropout_rate):
        super().__init__()
        layers=[]
        for i in range(num_hidden_layers):
            layers.append(nn.Linear(input_dim, neureaon_layers))
            layers.append(nn.BatchNorm1d(neureaon_layers))

        
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            input_dim=neureaon_layers
        layers.append(nn.Linear(neureaon_layers , output_dim))

        self.model=nn.Sequential(*layers)
    def forward(self,x):
        return self.model(x)



In [54]:
def objective(trial):

    # -------------------------
    # Hyperparameters (SEARCH SPACE)
    # -------------------------
    num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 5)
    neurons_per_layer = trial.suggest_int("neurons_per_layer", 8, 128, step=8)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    epochs=trial.suggest_int("epoches" ,10, 50 , step=10)
    dropout_rate=trial.suggest_float("dropout" , 0.1 ,0.5 ,step=0.1)
    batch_size=trial.suggest_categorical("batch_size" , [16,32,64,128])
    optimizer_name=trial.suggest_categorical("optimxer_name" ,[ 'Adam' ,'SGD' ,'RMSprop'])
    weight_decay=trial.suggest_float("weight_decay" , 1e-5 ,1e-3 ,log=True )


    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

    # -------------------------
    # Model initialization
    # -------------------------
    input_dim = 784
    output_dim = 10

    model = mynn(input_dim, output_dim, num_hidden_layers, neurons_per_layer , dropout_rate )
    model.to(device)

    # -------------------------
    # Loss & Optimizer
    # -------------------------
    criterion = nn.CrossEntropyLoss()
   

    if optimizer_name=='Adam':
         optimizer = optim.Adam(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )


    elif optimizer_name=='SGD':
        optimizer = optim.SGD(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )
    
    else:
         optimizer = optim.RMSprop(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )



    # -------------------------
    # Training loop
    # -------------------------
    model.train()

    for epoch in range(epochs):
        for batch_features, batch_labels in train_loader:

            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device)

            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # -------------------------
    # Evaluation loop
    # -------------------------
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for batch_features, batch_labels in test_loader:

            batch_features = batch_features.to(device)
            batch_labels = batch_labels.to(device)

            outputs = model(batch_features)
            _, predicted = torch.max(outputs, 1)

            total += batch_labels.size(0)
            correct += (predicted == batch_labels).sum().item()

    accuracy = correct / total

    return accuracy


In [55]:
import optuna 
study= optuna. create_study(direction='maximize')

[I 2025-12-27 08:58:18,548] A new study created in memory with name: no-name-cdf319e7-7ae4-4f4b-a032-fef59e79bf3a


In [56]:
study.optimize(objective , n_trials=10)

c:\Users\LOQ\Desktop\PyTorch\env_torch\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
[I 2025-12-27 08:58:30,598] Trial 0 finished with value: 0.82 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 48, 'learning_rate': 0.00196800982465174, 'epoches': 30, 'dropout': 0.2, 'batch_size': 16, 'optimxer_name': 'SGD', 'weight_decay': 0.0004057777252717032}. Best is trial 0 with value: 0.82.
[I 2025-12-27 08:58:41,822] Trial 1 finished with value: 0.7508333333333334 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 32, 'learning_rate': 0.0017509563281906959, 'epoches': 40, 'dropout': 0.30000000000000004, 'batch_size': 64, 'optimxer_name': 'RMSprop', 'weight_decay': 1.7603998914877532e-05}. Best is trial 0 with value: 0.82.
[I 2025-12-27 08:58:51,751] Trial 2 finished with value: 0.8275 and parameters: {'num_hidden_layer

In [57]:
study.best_value


0.85

In [58]:
study.best_params

{'num_hidden_layers': 1,
 'neurons_per_layer': 128,
 'learning_rate': 0.006559054569402451,
 'epoches': 40,
 'dropout': 0.4,
 'batch_size': 32,
 'optimxer_name': 'SGD',
 'weight_decay': 2.06590898558868e-05}

optuna x ML flow
  experiment tracking 
  trail=> log=> mlflow